# 05. 배치 이미지 처리 시스템

**담당**: Claude Opus  
**작성일**: 2025-10-24  
**목적**: 여러 이미지를 효율적으로 동시 분석하는 배치 처리 시스템

## 주요 기능
- 멀티프로세싱/멀티스레딩 기반 병렬 처리
- 동적 배치 크기 조정
- 우선순위 기반 큐 관리
- 메모리 효율적인 처리
- 실시간 진행 상황 모니터링

In [ ]:
# 필요 패키지 설치
import sys
!{sys.executable} -m pip install ultralytics opencv-python-headless numpy tqdm psutil

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import json
from typing import List, Dict, Optional, Tuple, Any, Union, Callable
from dataclasses import dataclass, field
from datetime import datetime, timedelta
import logging
from enum import Enum
import queue
import threading
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time
import psutil
from tqdm import tqdm
import pickle
import hashlib
from collections import deque, defaultdict
import warnings
warnings.filterwarnings('ignore')

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## 1. 배치 처리 데이터 클래스

In [ ]:
class ProcessingStatus(Enum):
    """처리 상태 열거형"""
    PENDING = "pending"
    PROCESSING = "processing"
    COMPLETED = "completed"
    FAILED = "failed"
    CANCELLED = "cancelled"

class Priority(Enum):
    """우선순위 레벨"""
    LOW = 3
    NORMAL = 2
    HIGH = 1
    URGENT = 0

@dataclass
class BatchItem:
    """배치 처리 항목"""
    id: str
    image_path: str
    image_data: Optional[np.ndarray] = None
    priority: Priority = Priority.NORMAL
    metadata: Dict = field(default_factory=dict)
    status: ProcessingStatus = ProcessingStatus.PENDING
    result: Optional[Any] = None
    error: Optional[str] = None
    processing_time: Optional[float] = None
    timestamp: datetime = field(default_factory=datetime.now)
    
    def __lt__(self, other):
        """우선순위 비교를 위한 메서드"""
        return self.priority.value < other.priority.value

@dataclass
class BatchResult:
    """배치 처리 결과"""
    batch_id: str
    total_items: int
    successful: int
    failed: int
    total_time: float
    average_time: float
    items: List[BatchItem]
    timestamp: datetime = field(default_factory=datetime.now)
    
    def to_dict(self) -> Dict:
        """딕셔너리 변환"""
        return {
            'batch_id': self.batch_id,
            'total_items': self.total_items,
            'successful': self.successful,
            'failed': self.failed,
            'total_time': self.total_time,
            'average_time': self.average_time,
            'timestamp': self.timestamp.isoformat()
        }

## 2. 리소스 모니터

In [ ]:
class ResourceMonitor:
    """시스템 리소스 모니터링"""
    
    def __init__(self):
        self.cpu_threshold = 90  # CPU 사용률 임계값 (%)
        self.memory_threshold = 85  # 메모리 사용률 임계값 (%)
        self.is_monitoring = False
        self.stats_history = deque(maxlen=100)
        
    def get_current_stats(self) -> Dict:
        """현재 시스템 상태 조회"""
        return {
            'cpu_percent': psutil.cpu_percent(interval=0.1),
            'memory_percent': psutil.virtual_memory().percent,
            'memory_available_gb': psutil.virtual_memory().available / (1024**3),
            'disk_usage_percent': psutil.disk_usage('/').percent,
            'timestamp': datetime.now()
        }
    
    def is_resource_available(self) -> bool:
        """리소스 사용 가능 여부 확인"""
        stats = self.get_current_stats()
        return (
            stats['cpu_percent'] < self.cpu_threshold and
            stats['memory_percent'] < self.memory_threshold
        )
    
    def get_optimal_batch_size(self, base_size: int = 32) -> int:
        """현재 리소스 기반 최적 배치 크기 계산"""
        stats = self.get_current_stats()
        
        # 메모리 기반 조정
        memory_factor = (100 - stats['memory_percent']) / 100
        
        # CPU 기반 조정
        cpu_factor = (100 - stats['cpu_percent']) / 100
        
        # 최적 배치 크기 계산
        optimal_size = int(base_size * min(memory_factor, cpu_factor))
        
        return max(1, min(optimal_size, base_size))
    
    def get_optimal_workers(self) -> int:
        """최적 워커 수 계산"""
        cpu_count = mp.cpu_count()
        stats = self.get_current_stats()
        
        # CPU 사용률에 따라 워커 수 조정
        if stats['cpu_percent'] > 80:
            return max(1, cpu_count // 2)
        elif stats['cpu_percent'] > 60:
            return max(2, cpu_count - 2)
        else:
            return cpu_count
    
    def start_monitoring(self, callback: Optional[Callable] = None):
        """백그라운드 모니터링 시작"""
        self.is_monitoring = True
        
        def monitor_loop():
            while self.is_monitoring:
                stats = self.get_current_stats()
                self.stats_history.append(stats)
                
                if callback:
                    callback(stats)
                
                time.sleep(1)
        
        thread = threading.Thread(target=monitor_loop, daemon=True)
        thread.start()
    
    def stop_monitoring(self):
        """모니터링 중지"""
        self.is_monitoring = False

## 3. 배치 큐 관리자

In [ ]:
class BatchQueue:
    """우선순위 기반 배치 큐"""
    
    def __init__(self, max_size: Optional[int] = None):
        self.queue = queue.PriorityQueue(maxsize=max_size or 0)
        self.processing_items = {}
        self.completed_items = {}
        self.failed_items = {}
        self.lock = threading.Lock()
        self.stats = defaultdict(int)
        
    def add_item(self, item: BatchItem):
        """항목 추가"""
        self.queue.put((item.priority.value, item.timestamp, item))
        self.stats['total_added'] += 1
        logger.debug(f"항목 추가: {item.id} (우선순위: {item.priority.name})")
    
    def add_batch(self, items: List[BatchItem]):
        """여러 항목 일괄 추가"""
        for item in items:
            self.add_item(item)
        logger.info(f"{len(items)}개 항목 큐에 추가")
    
    def get_item(self, timeout: Optional[float] = None) -> Optional[BatchItem]:
        """항목 가져오기"""
        try:
            _, _, item = self.queue.get(timeout=timeout)
            with self.lock:
                item.status = ProcessingStatus.PROCESSING
                self.processing_items[item.id] = item
            return item
        except queue.Empty:
            return None
    
    def get_batch(self, batch_size: int, timeout: float = 1.0) -> List[BatchItem]:
        """배치 단위로 항목 가져오기"""
        batch = []
        deadline = time.time() + timeout
        
        while len(batch) < batch_size and time.time() < deadline:
            remaining_time = deadline - time.time()
            if remaining_time <= 0:
                break
            
            item = self.get_item(timeout=min(0.1, remaining_time))
            if item:
                batch.append(item)
        
        return batch
    
    def mark_completed(self, item: BatchItem):
        """항목을 완료로 표시"""
        with self.lock:
            item.status = ProcessingStatus.COMPLETED
            if item.id in self.processing_items:
                del self.processing_items[item.id]
            self.completed_items[item.id] = item
            self.stats['completed'] += 1
    
    def mark_failed(self, item: BatchItem, error: str):
        """항목을 실패로 표시"""
        with self.lock:
            item.status = ProcessingStatus.FAILED
            item.error = error
            if item.id in self.processing_items:
                del self.processing_items[item.id]
            self.failed_items[item.id] = item
            self.stats['failed'] += 1
    
    def get_stats(self) -> Dict:
        """통계 정보 반환"""
        with self.lock:
            return {
                'pending': self.queue.qsize(),
                'processing': len(self.processing_items),
                'completed': len(self.completed_items),
                'failed': len(self.failed_items),
                'total': self.stats['total_added']
            }
    
    def is_empty(self) -> bool:
        """큐가 비어있는지 확인"""
        return self.queue.empty() and len(self.processing_items) == 0

## 4. 이미지 프로세서

In [ ]:
class ImageProcessor:
    """개별 이미지 처리 클래스"""
    
    def __init__(self, model_func: Optional[Callable] = None):
        self.model_func = model_func or self._default_process
        
    def _default_process(self, image: np.ndarray) -> Dict:
        """기본 처리 함수 (테스트용)"""
        # 실제로는 YOLO11 모델 추론이 들어갈 위치
        time.sleep(0.1)  # 처리 시뮬레이션
        return {
            'detections': [],
            'confidence': 0.0,
            'processing_time': 0.1
        }
    
    def process_image(self, item: BatchItem) -> BatchItem:
        """단일 이미지 처리"""
        start_time = time.time()
        
        try:
            # 이미지 로드
            if item.image_data is None:
                image = cv2.imread(item.image_path)
                if image is None:
                    raise ValueError(f"이미지 로드 실패: {item.image_path}")
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            else:
                image = item.image_data
            
            # 모델 처리
            result = self.model_func(image)
            
            # 결과 저장
            item.result = result
            item.processing_time = time.time() - start_time
            item.status = ProcessingStatus.COMPLETED
            
        except Exception as e:
            item.error = str(e)
            item.status = ProcessingStatus.FAILED
            item.processing_time = time.time() - start_time
            logger.error(f"이미지 처리 실패 {item.id}: {e}")
        
        return item
    
    def process_batch(self, items: List[BatchItem]) -> List[BatchItem]:
        """배치 단위 처리"""
        results = []
        for item in items:
            processed = self.process_image(item)
            results.append(processed)
        return results

## 5. 배치 처리 엔진

In [ ]:
class BatchProcessingEngine:
    """메인 배치 처리 엔진"""
    
    def __init__(self, 
                 model_func: Optional[Callable] = None,
                 num_workers: Optional[int] = None,
                 batch_size: int = 32,
                 use_multiprocessing: bool = False):
        
        self.processor = ImageProcessor(model_func)
        self.resource_monitor = ResourceMonitor()
        self.batch_queue = BatchQueue()
        
        # 워커 설정
        self.num_workers = num_workers or self.resource_monitor.get_optimal_workers()
        self.batch_size = batch_size
        self.use_multiprocessing = use_multiprocessing
        
        # 실행 제어
        self.is_running = False
        self.executor = None
        self.workers = []
        
        # 통계
        self.start_time = None
        self.processed_count = 0
        
        logger.info(f"배치 처리 엔진 초기화 (워커: {self.num_workers}, 배치 크기: {self.batch_size})")
    
    def add_images(self, image_paths: List[str], priority: Priority = Priority.NORMAL):
        """이미지 추가"""
        items = []
        for i, path in enumerate(image_paths):
            item = BatchItem(
                id=f"{datetime.now().timestamp()}_{i}",
                image_path=path,
                priority=priority
            )
            items.append(item)
        
        self.batch_queue.add_batch(items)
        logger.info(f"{len(items)}개 이미지 추가 (우선순위: {priority.name})")
    
    def _worker_thread(self, worker_id: int):
        """워커 스레드"""
        logger.info(f"워커 {worker_id} 시작")
        
        while self.is_running:
            # 리소스 체크
            if not self.resource_monitor.is_resource_available():
                time.sleep(1)
                continue
            
            # 동적 배치 크기 조정
            current_batch_size = self.resource_monitor.get_optimal_batch_size(self.batch_size)
            
            # 배치 가져오기
            batch = self.batch_queue.get_batch(current_batch_size, timeout=1.0)
            
            if not batch:
                if self.batch_queue.is_empty():
                    time.sleep(0.1)
                continue
            
            # 배치 처리
            logger.debug(f"워커 {worker_id}: {len(batch)}개 항목 처리 시작")
            
            try:
                processed = self.processor.process_batch(batch)
                
                for item in processed:
                    if item.status == ProcessingStatus.COMPLETED:
                        self.batch_queue.mark_completed(item)
                    else:
                        self.batch_queue.mark_failed(item, item.error or "Unknown error")
                    
                    self.processed_count += 1
                    
            except Exception as e:
                logger.error(f"워커 {worker_id} 배치 처리 오류: {e}")
                for item in batch:
                    self.batch_queue.mark_failed(item, str(e))
        
        logger.info(f"워커 {worker_id} 종료")
    
    def start(self):
        """처리 시작"""
        if self.is_running:
            logger.warning("이미 실행 중")
            return
        
        self.is_running = True
        self.start_time = time.time()
        self.processed_count = 0
        
        # 리소스 모니터링 시작
        self.resource_monitor.start_monitoring()
        
        # 워커 스레드 시작
        if self.use_multiprocessing:
            self.executor = ProcessPoolExecutor(max_workers=self.num_workers)
        else:
            self.executor = ThreadPoolExecutor(max_workers=self.num_workers)
        
        for i in range(self.num_workers):
            worker = threading.Thread(target=self._worker_thread, args=(i,))
            worker.daemon = True
            worker.start()
            self.workers.append(worker)
        
        logger.info(f"배치 처리 시작 ({self.num_workers}개 워커)")
    
    def stop(self, wait: bool = True):
        """처리 중지"""
        if not self.is_running:
            return
        
        logger.info("배치 처리 중지 요청")
        self.is_running = False
        
        # 워커 종료 대기
        if wait:
            for worker in self.workers:
                worker.join(timeout=5)
        
        # Executor 종료
        if self.executor:
            self.executor.shutdown(wait=wait)
        
        # 모니터링 중지
        self.resource_monitor.stop_monitoring()
        
        # 최종 통계
        if self.start_time:
            elapsed = time.time() - self.start_time
            logger.info(f"배치 처리 완료: {self.processed_count}개 처리, 소요시간: {elapsed:.2f}초")
    
    def wait_for_completion(self, timeout: Optional[float] = None):
        """모든 작업 완료 대기"""
        start = time.time()
        
        while not self.batch_queue.is_empty():
            if timeout and (time.time() - start) > timeout:
                logger.warning("완료 대기 타임아웃")
                return False
            time.sleep(0.1)
        
        return True
    
    def get_results(self) -> BatchResult:
        """처리 결과 반환"""
        stats = self.batch_queue.get_stats()
        elapsed = time.time() - self.start_time if self.start_time else 0
        
        all_items = (
            list(self.batch_queue.completed_items.values()) +
            list(self.batch_queue.failed_items.values())
        )
        
        return BatchResult(
            batch_id=f"batch_{datetime.now().timestamp()}",
            total_items=stats['total'],
            successful=stats['completed'],
            failed=stats['failed'],
            total_time=elapsed,
            average_time=elapsed / max(1, self.processed_count),
            items=all_items
        )
    
    def get_progress(self) -> Dict:
        """진행 상황 조회"""
        stats = self.batch_queue.get_stats()
        elapsed = time.time() - self.start_time if self.start_time else 0
        
        return {
            **stats,
            'elapsed_time': elapsed,
            'items_per_second': self.processed_count / max(1, elapsed),
            'estimated_remaining': self._estimate_remaining_time(stats)
        }
    
    def _estimate_remaining_time(self, stats: Dict) -> float:
        """남은 시간 예측"""
        if self.processed_count == 0:
            return -1
        
        elapsed = time.time() - self.start_time if self.start_time else 0
        rate = self.processed_count / max(1, elapsed)
        remaining = stats['pending'] + stats['processing']
        
        return remaining / max(0.01, rate)

## 6. 진행 상황 모니터

In [ ]:
class ProgressMonitor:
    """실시간 진행 상황 모니터링"""
    
    def __init__(self, engine: BatchProcessingEngine):
        self.engine = engine
        self.is_monitoring = False
        self.monitor_thread = None
        
    def start_monitoring(self, interval: float = 1.0, use_tqdm: bool = True):
        """모니터링 시작"""
        self.is_monitoring = True
        
        def monitor_loop():
            if use_tqdm:
                stats = self.engine.batch_queue.get_stats()
                pbar = tqdm(total=stats['total'], desc="배치 처리")
                last_completed = 0
                
                while self.is_monitoring:
                    progress = self.engine.get_progress()
                    
                    # tqdm 업데이트
                    completed = progress['completed'] + progress['failed']
                    pbar.update(completed - last_completed)
                    last_completed = completed
                    
                    # 상태 표시
                    pbar.set_postfix({
                        '대기': progress['pending'],
                        '처리중': progress['processing'],
                        '완료': progress['completed'],
                        '실패': progress['failed'],
                        '속도': f"{progress['items_per_second']:.1f}/s"
                    })
                    
                    if self.engine.batch_queue.is_empty():
                        break
                    
                    time.sleep(interval)
                
                pbar.close()
            else:
                while self.is_monitoring:
                    progress = self.engine.get_progress()
                    self._print_progress(progress)
                    
                    if self.engine.batch_queue.is_empty():
                        break
                    
                    time.sleep(interval)
        
        self.monitor_thread = threading.Thread(target=monitor_loop, daemon=True)
        self.monitor_thread.start()
    
    def stop_monitoring(self):
        """모니터링 중지"""
        self.is_monitoring = False
        if self.monitor_thread:
            self.monitor_thread.join(timeout=2)
    
    def _print_progress(self, progress: Dict):
        """진행 상황 출력"""
        print(f"\r[진행 상황] 대기: {progress['pending']} | "
              f"처리중: {progress['processing']} | "
              f"완료: {progress['completed']} | "
              f"실패: {progress['failed']} | "
              f"속도: {progress['items_per_second']:.1f}/s", end="")

## 7. 사용 예제

In [ ]:
# 테스트용 이미지 처리 함수
def mock_yolo_inference(image: np.ndarray) -> Dict:
    """YOLO 추론 시뮬레이션"""
    time.sleep(np.random.uniform(0.05, 0.15))  # 처리 시간 시뮬레이션
    
    # 가짜 탐지 결과 생성
    num_detections = np.random.randint(0, 5)
    detections = []
    
    for i in range(num_detections):
        detections.append({
            'class': np.random.choice(['wheat', 'corn', 'rice', 'soybean']),
            'confidence': np.random.uniform(0.5, 0.99),
            'bbox': [np.random.randint(0, 640) for _ in range(4)]
        })
    
    return {
        'detections': detections,
        'num_objects': num_detections,
        'inference_time': time.time()
    }

# 예제 1: 기본 배치 처리
print("=== 예제 1: 기본 배치 처리 ===")
print("배치 처리 엔진 생성...")

engine = BatchProcessingEngine(
    model_func=mock_yolo_inference,
    num_workers=4,
    batch_size=16,
    use_multiprocessing=False
)

# 테스트 이미지 경로 생성 (실제로는 실제 경로 사용)
test_images = [f"test_image_{i}.jpg" for i in range(50)]

# 이미지 추가 (우선순위별)
engine.add_images(test_images[:10], priority=Priority.URGENT)
engine.add_images(test_images[10:30], priority=Priority.NORMAL)
engine.add_images(test_images[30:], priority=Priority.LOW)

print(f"총 {len(test_images)}개 이미지 추가 완료")
print("처리 시작...\n")

# 처리 시작
engine.start()

# 진행 상황 모니터링
monitor = ProgressMonitor(engine)
monitor.start_monitoring(use_tqdm=True)

# 완료 대기
engine.wait_for_completion(timeout=30)

# 결과 가져오기
results = engine.get_results()

# 처리 중지
engine.stop()
monitor.stop_monitoring()

# 결과 출력
print("\n\n=== 처리 결과 ===")
print(f"총 처리: {results.total_items}개")
print(f"성공: {results.successful}개")
print(f"실패: {results.failed}개")
print(f"총 소요시간: {results.total_time:.2f}초")
print(f"평균 처리시간: {results.average_time:.3f}초/이미지")
print(f"처리 속도: {results.total_items/max(1, results.total_time):.1f} 이미지/초")

## 8. 고급 기능: 적응적 배치 처리

In [ ]:
class AdaptiveBatchProcessor:
    """리소스 기반 적응적 배치 처리"""
    
    def __init__(self, engine: BatchProcessingEngine):
        self.engine = engine
        self.performance_history = deque(maxlen=100)
        self.optimal_batch_size = engine.batch_size
        self.optimal_workers = engine.num_workers
        
    def auto_tune(self, test_images: List[str], duration: float = 30):
        """자동 성능 튜닝"""
        print("자동 튜닝 시작...")
        
        test_configs = [
            (8, 2),   # batch_size, num_workers
            (16, 4),
            (32, 4),
            (32, 8),
            (64, 8),
        ]
        
        best_throughput = 0
        best_config = (self.optimal_batch_size, self.optimal_workers)
        
        for batch_size, num_workers in test_configs:
            print(f"\n테스트: 배치크기={batch_size}, 워커={num_workers}")
            
            # 엔진 재구성
            test_engine = BatchProcessingEngine(
                model_func=self.engine.processor.model_func,
                num_workers=num_workers,
                batch_size=batch_size
            )
            
            # 테스트 실행
            test_engine.add_images(test_images[:20])  # 샘플 사용
            test_engine.start()
            
            # 일정 시간 실행
            time.sleep(min(duration/len(test_configs), 5))
            
            # 성능 측정
            progress = test_engine.get_progress()
            throughput = progress['items_per_second']
            
            print(f"  처리량: {throughput:.2f} 이미지/초")
            
            if throughput > best_throughput:
                best_throughput = throughput
                best_config = (batch_size, num_workers)
            
            test_engine.stop(wait=False)
        
        self.optimal_batch_size, self.optimal_workers = best_config
        
        print(f"\n최적 설정: 배치크기={self.optimal_batch_size}, "
              f"워커={self.optimal_workers}, "
              f"처리량={best_throughput:.2f} 이미지/초")
        
        return best_config
    
    def process_with_adaptation(self, image_paths: List[str]):
        """적응적 처리"""
        # 최적 설정으로 엔진 재구성
        self.engine.batch_size = self.optimal_batch_size
        self.engine.num_workers = self.optimal_workers
        
        # 처리 시작
        self.engine.add_images(image_paths)
        self.engine.start()
        
        # 동적 조정
        def adjust_parameters():
            while self.engine.is_running:
                stats = self.engine.resource_monitor.get_current_stats()
                
                # CPU 사용률에 따른 조정
                if stats['cpu_percent'] > 90:
                    self.engine.batch_size = max(1, self.engine.batch_size // 2)
                elif stats['cpu_percent'] < 50:
                    self.engine.batch_size = min(64, self.engine.batch_size * 2)
                
                time.sleep(5)
        
        adjust_thread = threading.Thread(target=adjust_parameters, daemon=True)
        adjust_thread.start()
        
        return self.engine

## 9. 결과 내보내기

In [ ]:
class ResultExporter:
    """배치 처리 결과 내보내기"""
    
    @staticmethod
    def export_to_json(result: BatchResult, output_path: str):
        """JSON 형식으로 내보내기"""
        data = result.to_dict()
        
        # 각 항목의 상세 정보 추가
        data['items'] = []
        for item in result.items:
            item_data = {
                'id': item.id,
                'path': item.image_path,
                'status': item.status.value,
                'processing_time': item.processing_time,
                'error': item.error,
                'result': item.result
            }
            data['items'].append(item_data)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, default=str)
        
        logger.info(f"결과 저장: {output_path}")
    
    @staticmethod
    def export_to_csv(result: BatchResult, output_path: str):
        """CSV 형식으로 내보내기"""
        import csv
        
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            fieldnames = ['id', 'path', 'status', 'processing_time', 
                         'num_detections', 'error']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            for item in result.items:
                row = {
                    'id': item.id,
                    'path': item.image_path,
                    'status': item.status.value,
                    'processing_time': item.processing_time,
                    'num_detections': len(item.result.get('detections', [])) if item.result else 0,
                    'error': item.error or ''
                }
                writer.writerow(row)
        
        logger.info(f"CSV 결과 저장: {output_path}")
    
    @staticmethod
    def generate_summary_report(result: BatchResult) -> str:
        """요약 보고서 생성"""
        report = []
        report.append("=" * 50)
        report.append("배치 처리 요약 보고서")
        report.append("=" * 50)
        report.append(f"배치 ID: {result.batch_id}")
        report.append(f"처리 시간: {result.timestamp}")
        report.append(f"총 항목: {result.total_items}")
        report.append(f"성공: {result.successful} ({result.successful/max(1, result.total_items)*100:.1f}%)")
        report.append(f"실패: {result.failed} ({result.failed/max(1, result.total_items)*100:.1f}%)")
        report.append(f"총 소요시간: {result.total_time:.2f}초")
        report.append(f"평균 처리시간: {result.average_time:.3f}초")
        report.append(f"처리 속도: {result.total_items/max(1, result.total_time):.1f} 이미지/초")
        
        if result.failed > 0:
            report.append("\n실패 항목:")
            for item in result.items:
                if item.status == ProcessingStatus.FAILED:
                    report.append(f"  - {item.image_path}: {item.error}")
        
        return "\n".join(report)

# 사용 예제
print("=== 결과 내보내기 예제 ===")

# 가짜 결과 생성 (실제 처리 후 사용)
if 'results' in locals():
    # 요약 보고서 출력
    report = ResultExporter.generate_summary_report(results)
    print(report)
    
    # JSON으로 저장 (실제 사용시)
    # ResultExporter.export_to_json(results, "batch_results.json")
    
    # CSV로 저장 (실제 사용시)
    # ResultExporter.export_to_csv(results, "batch_results.csv")

## 10. 통합 사용 예제

In [ ]:
def complete_batch_processing_pipeline():
    """완전한 배치 처리 파이프라인 예제"""
    
    print("🚀 통합 배치 처리 파이프라인 시작\n")
    
    # 1. 엔진 생성
    print("[1/5] 배치 처리 엔진 초기화...")
    engine = BatchProcessingEngine(
        model_func=mock_yolo_inference,
        num_workers=None,  # 자동 설정
        batch_size=32
    )
    
    # 2. 테스트 이미지 준비
    print("[2/5] 이미지 준비...")
    test_images = [f"drone_image_{i:04d}.jpg" for i in range(100)]
    
    # 3. 자동 튜닝 (선택적)
    print("[3/5] 성능 자동 튜닝...")
    adaptive = AdaptiveBatchProcessor(engine)
    # adaptive.auto_tune(test_images[:20], duration=10)  # 실제 사용시 활성화
    
    # 4. 이미지 처리
    print("[4/5] 배치 처리 실행...")
    
    # 우선순위별로 이미지 추가
    engine.add_images(test_images[:20], Priority.URGENT)   # 긴급
    engine.add_images(test_images[20:70], Priority.NORMAL) # 일반
    engine.add_images(test_images[70:], Priority.LOW)      # 낮음
    
    # 처리 시작
    engine.start()
    
    # 진행 상황 모니터링
    monitor = ProgressMonitor(engine)
    monitor.start_monitoring(use_tqdm=True)
    
    # 완료 대기
    engine.wait_for_completion(timeout=60)
    
    # 5. 결과 처리
    print("\n[5/5] 결과 처리...")
    results = engine.get_results()
    
    # 엔진 정리
    engine.stop()
    monitor.stop_monitoring()
    
    # 결과 요약
    print("\n" + "=" * 50)
    print("📊 최종 결과")
    print("=" * 50)
    print(f"✅ 성공: {results.successful}/{results.total_items}")
    print(f"❌ 실패: {results.failed}/{results.total_items}")
    print(f"⏱️  총 시간: {results.total_time:.1f}초")
    print(f"🚀 처리 속도: {results.total_items/max(1, results.total_time):.1f} 이미지/초")
    print(f"📈 평균 시간: {results.average_time*1000:.1f}ms/이미지")
    
    # 결과 저장 (실제 사용시)
    # ResultExporter.export_to_json(results, "results/batch_results.json")
    # ResultExporter.export_to_csv(results, "results/batch_results.csv")
    
    print("\n✨ 배치 처리 완료!")
    
    return results

# 파이프라인 실행
final_results = complete_batch_processing_pipeline()

## 11. 다음 모듈 연동 준비

In [ ]:
print("\n🔗 다음 모듈 연동 정보\n")

print("배치 처리 시스템이 완성되었습니다!\n")

print("이 모듈은 다음과 같이 연동됩니다:\n")

print("1. ← Todo 2 (입력 처리):")
print("   - ProcessedInput 객체를 BatchItem으로 변환")
print("   - 메타데이터 기반 우선순위 설정")

print("\n2. → Todo 6 (결과 저장):")
print("   - BatchResult 객체 전달")
print("   - 처리된 이미지와 탐지 결과 저장")

print("\n3. → Todo 7 (스케줄링):")
print("   - 정기적 배치 작업 실행")
print("   - 리소스 모니터링 정보 활용")

print("\n4. → Todo 8 (시각화):")
print("   - 처리 통계 시각화")
print("   - 실시간 진행 상황 대시보드")

print("\n주요 특징:")
print("✓ 우선순위 기반 처리")
print("✓ 동적 리소스 관리")
print("✓ 병렬 처리 최적화")
print("✓ 실시간 진행 모니터링")
print("✓ 자동 성능 튜닝")

print("\n배치 처리 시스템 개발 완료! ✨")